<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/assignments/assignment_yourname_t81_559_class4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative AI
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/index.html)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-559/).

**Module 4 Assignment: Chat Memory**

**Student Name: Your Name**

# Assignment Instructions

A [file](https://data.heatonresearch.com/data/t81-559/assignments/transcript4.csv) is provided that contains a transcript with two chatbots. Responses should always be a single sentence. The file is shown here:

|target|prompt|
|--|--|
|chat1|Hello my name is John.|
|chat2|Hello my name is Jane|
|chat1|My favorite color is blue.|
|chat2|My favorite color is yellow.|
|chat1|What is my name and favorite color.|
|chat2|What is my name?|

Write a program that creates two chatbots, each should have their own memory. Send these transcript items to the indicated chatbot and record their responses in your output file.

Your output should look like something this:

|target|response|
|--|--|
|chat1|Hi John.|
|chat2|Hello Jane.|
|chat1|Thank you for this information.|
|chat2|Thank you for this information.|
|chat1|Hi John, your name is John and your favorite color is blue.|
|chat2|Hey Jane, your name is Jane.|

Note that the LLM will not give this exact output, but it should be similar.



# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process. Running the following code will map your GDrive to ```/content/drive```.

In [ ]:
import os

try:
  from google.colab import drive, userdata
  drive.mount('/content/drive', force_remount=True)
  COLAB = True
  print("Note: using Google CoLab")
except:
  print("Note: not using Google CoLab")
  COLAB = False

# Assignment Submission Key - Was sent you first week of class.
# If you are in both classes, this is the same key.
if COLAB:
  # For Colab, add to your "Secrets" (key icon at the left)
  key = userdata.get('T81_559_KEY')
else:
  # If not colab, enter your key here, or use an environment variable.
  # (this is only an example key, use yours)
  key = ""

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Mounted at /content/drive
Note: using Google CoLab
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33


# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems.

**It is unlikely that should need to modify this function.**

In [ ]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io
from typing import List, Union

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# course - The course that you are in, currently t81-558 or t81-559.
# no - The assignment class number, should be 1 through 10.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.

def submit(
    data: List[Union[pd.DataFrame, PIL.Image.Image]],
    key: str,
    course: str,
    no: int,
    source_file: str = None
) -> None:
    if source_file is None and '__file__' not in globals():
        raise Exception("Must specify a filename when in a Jupyter notebook.")
    if source_file is None:
        source_file = __file__

    suffix = f'_class{no}'
    if suffix not in source_file:
        raise Exception(f"{suffix} must be part of the filename.")

    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb', '.py']:
        raise Exception(f"Source file is {ext}; must be .py or .ipynb")

    with open(source_file, "rb") as file:
        encoded_python = base64.b64encode(file.read()).decode('ascii')

    payload = []
    for item in data:
        if isinstance(item, PIL.Image.Image):
            buffered = io.BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG': base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif isinstance(item, pd.DataFrame):
            payload.append({'CSV': base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
        else:
            raise ValueError(f"Unsupported data type: {type(item)}")

    response = requests.post(
        "https://api.heatonresearch.com/wu/submit",
        headers={'x-api-key': key},
        json={
            'payload': payload,
            'assignment': no,
            'course': course,
            'ext': ext,
            'py': encoded_python
        }
    )

    if response.status_code == 200:
        print(f"Success: {response.text}")
    else:
        print(f"Failure: {response.text}")

# Assignment #4 Sample Code

The following code provides a starting point for this assignment.

In [ ]:
import os
import pandas as pd
from scipy.stats import zscore
import string
from langchain.prompts import ChatPromptTemplate

# You must identify your source file.  (modify for your local setup)
# file="/content/drive/My Drive/Colab Notebooks/assignment_yourname_t81_559_class4.ipynb"  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_559_deep_learning\\assignments\\assignment_yourname_t81_559_class4.ipynb'  # Windows
# file='/Users/jheaton/projects/t81_559_deep_learning/assignments/assignment_yourname_t81_559_class4.ipynb'  # Mac/Linux
file="/content/drive/My Drive/Colab Notebooks/assignment_ZhijiangLi_t81_559_class4.ipynb"

# Begin assignment

df = pd.read_csv("https://data.heatonresearch.com/data/t81-559/assignments/transcript4.csv")
df

,target,prompt
0,chat1,Hello my name is John.
1,chat2,Hello my name is Jane
2,chat1,My favorite color is blue.
3,chat2,My favorite color is yellow.
4,chat1,What is my name and favorite color.
5,chat2,What is my name?


In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown
import pickle

DEFAULT_TEMPLATE = """You are a helpful assistant. Format answers with text only.
Answer in a single sentence.

Current conversation:
{history}
Human: {input}
AI:"""

MODEL = 'gpt-4o-mini'

class ChatBot:
    def __init__(self, llm_chat, llm_summary, template):
        """
        Initializes the ChatBot with language models and a template for conversation.

        :param llm_chat: A large language model for handling chat responses.
        :param llm_summary: A large language model for summarizing conversations.
        :param template: A string template defining the conversation structure.
        """
        self.llm_chat = llm_chat
        self.llm_summary = llm_summary
        self.template = template
        self.prompt_template = PromptTemplate(input_variables=["history", "input"], template=self.template)

        # Initialize memory and conversation chain
        self.memory = ConversationSummaryMemory(llm=self.llm_summary)
        self.conversation = ConversationChain(
            prompt=self.prompt_template,
            llm=self.llm_chat,
            memory=self.memory,
            verbose=False
        )

        self.history = []

    def converse(self, prompt):
        """
        Processes a conversation prompt and updates the internal history and memory.

        :param prompt: The input prompt from the user.
        :return: The generated response from the language model.
        """
        self.history.append([self.memory.buffer, prompt])
        output = self.conversation.invoke(prompt)
        return output['response']

    def chat(self, prompt):
        """
        Handles the full cycle of receiving a prompt, processing it, and displaying the result.

        :param prompt: The input prompt from the user.
        """
        print(f"Human: {prompt}")
        output = self.converse(prompt)
        display_markdown(output, raw=True)

    def print_memory(self):
        """
        Displays the current state of the conversation memory.
        """
        print("**Memory:")
        print(self.memory.buffer)

    def clear_memory(self):
        """
        Clears the conversation memory.
        """
        self.memory.clear()

    def undo(self):
        """
        Reverts the conversation memory to the state before the last interaction.
        """
        if len(self.history) > 0:
            self.memory.buffer = self.history.pop()[0]
        else:
            print("Nothing to undo.")

    def regenerate(self):
        """
        Re-executes the last undone interaction, effectively redoing an undo operation.
        """
        if len(self.history) > 0:
            self.memory.buffer, prompt = self.history.pop()
            self.chat(prompt)
        else:
            print("Nothing to regenerate.")

    def save_history(self, file_path):
        """
        Saves the conversation history to a file using pickle.

        :param file_path: The file path where the history should be saved.
        """
        with open(file_path, 'wb') as f:
            pickle.dump(self.history, f)

    def load_history(self, file_path):
        """
        Loads the conversation history from a file using pickle.

        :param file_path: The file path from which to load the history.
        """
        with open(file_path, 'rb') as f:
            self.history = pickle.load(f)
            # Optionally reset the memory based on the last saved state
            if self.history:
                self.memory.buffer = self.history[-1][0]

In [ ]:
## ... continue your code...
llm = ChatOpenAI(model=MODEL, temperature=0.3, n=1)
chatbot1 = ChatBot(llm, llm, DEFAULT_TEMPLATE)
chatbot2 = ChatBot(llm, llm, DEFAULT_TEMPLATE)

# Process transcript and store responses
responses = []

for _, row in df.iterrows():
    target, prompt = row["target"], row["prompt"]
    if target == "chat1":
        response = chatbot1.converse(prompt)
    elif target == "chat2":
        response = chatbot2.converse(prompt)
    else:
        response = "Invalid chatbot target"

    responses.append({"target": target, "response": response})

# Save output as a CSV file
df_submit = pd.DataFrame(responses)
print(df_submit.head(10))

## Submit assignment

# Submit
submit(source_file=file,data=[df_submit],course='t81-559',key=key,no=4)

   target                                           response
0   chat1                            Nice to meet you, John!
1   chat2                      Hello Jane, nice to meet you!
2   chat1  That's a great choice; blue is often associate...
3  chat2                              Invalid chatbot target
4   chat1  Your name is John and your favorite color is b...
5   chat2                                 Your name is Jane.
Success: Submitted Assignment 4 (t81-559) for l.zhijiang:
You have submitted this assignment 4 times. (this is fine)
Based on the provided information, here's the evaluation of the STUDENT CODE and STUDENT CSV DATA:
Item 1: Yes. The STUDENT CSV DATA has the same number of lines (6) as the EXPECTED CSV OUTPUT.
Item 2: Yes. The STUDENT CSV DATA contains both target and response columns.
Item 3: Yes. The PROVIDED CSV INPUT could have generated the outputs in STUDENT CSV DATA. The responses are consistent with the prompts given in the input.
Item 4: Yes. The STUDENT CODE